<a href="https://colab.research.google.com/github/miller00315/crew_ai_studies/blob/main/crew_ai_correct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install crewai crewai_tools langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.1/252.1 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.5/548.5 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.4/211.4 kB 16.3 MB/s eta

In [2]:
from crewai import LLM

deepseek_reasoner_r1 = LLM(
    model="openrouter/deepseek/deepseek-r1",
    base_url="https://openrouter.ai/api/v1",
    api_key='',
)

In [13]:
from crewai import Agent, Task, Crew

In [5]:
planner = Agent(
    llm=deepseek_reasoner_r1,
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

In [7]:
writer = Agent(
    llm=deepseek_reasoner_r1,
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

In [8]:
editor = Agent(
    llm=deepseek_reasoner_r1,
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

**Preparação das tarefas:**

# Nova seção

In [9]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

In [10]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

In [11]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

In [17]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

In [18]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 
[2025-03-11 00:35:28][🚀 CREW 'CREW' STARTED, 9D285DA7-9831-4A79-8A6B-4A34946C75A4]: 2025-03-11 00:35:28.727855
 
[2025-03-11 00:35:28][📋 TASK STARTED: 1. PRIORITIZE THE LATEST TRENDS, KEY PLAYERS, AND NOTEWORTHY NEWS ON ARTIFICIAL INTELLIGENCE.
2. IDENTIFY THE TARGET AUDIENCE, CONSIDERING THEIR INTERESTS AND PAIN POINTS.
3. DEVELOP A DETAILED CONTENT OUTLINE INCLUDING AN INTRODUCTION, KEY POINTS, AND A CALL TO ACTION.
4. INCLUDE SEO KEYWORDS AND RELEVANT DATA OR SOURCES.]: 2025-03-11 00:35:28.747942
 
[2025-03-11 00:35:28][🤖 AGENT 'CONTENT PLANNER' STARTED TASK]: 2025-03-11 00:35:28.749187
# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.
 
[2025-03-11 00:35:28][🤖 LLM CALL STARTED

In [22]:
from IPython.display import Markdown

Markdown(str(result))

# Artificial Intelligence in 2023: Trends, Players, and Ethical Frontiers  

## Introduction  
**AI is reshaping industries at unprecedented speed—here’s what you need to know.** Once confined to academic papers and sci-fi movies, artificial intelligence has surged into the mainstream, revolutionizing everything from healthcare diagnostics to creative workflows. In 2023, advancements like ChatGPT and DALL-E 3 dominate headlines, but the real story lies in how businesses, developers, and policymakers are navigating this transformative era.  

Whether you’re a tech professional optimizing machine learning models or a CEO weighing AI adoption, this article unpacks the latest trends, key innovators, and ethical frameworks shaping the future. With 70% of healthcare providers planning AI adoption by 2025 (McKinsey) and generative AI markets projected to reach $42.6 billion this year (Gartner), understanding these shifts isn’t optional—it’s essential.  

---

## Section 1: Latest Trends in AI (2023)  
### **Generative AI Breaks Creative Boundaries**  
Generative AI isn’t just automating tasks—it’s redefining creativity. OpenAI’s GPT-4 and DALL-E 3 now produce human-like text and hyper-realistic images, while multimodal models like Google’s Gemini blend text, audio, and vision for richer outputs. Developers leverage these tools for code generation, while marketers use them to draft personalized campaigns at scale.  

However, scalability brings challenges. Training models like GPT-4 consumed 1,287 MWh of energy in 2023 (Stanford AI Index), sparking debates about sustainability. As the EU AI Act mandates transparency reports, companies must balance innovation with environmental responsibility.  

### **AI in Healthcare: Saving Time, Saving Lives**  
From drug discovery to diagnostics, AI accelerates healthcare innovation. DeepMind’s AlphaFold has predicted 3D structures for over 200 million proteins, unlocking new avenues for disease research. Institutions like the Mayo Clinic deploy AI to analyze medical images with 95% accuracy, reducing diagnostic delays.  

These advancements come with caveats. While AI can address staffing shortages, overreliance on algorithms without human oversight risks misdiagnosis. The World Health Organization now urges "human-in-the-loop" systems for critical care decisions.  

### **The Rise of Ethical AI and Edge Computing**  
As AI scales, privacy and sustainability concerns grow. The Biden Administration’s Executive Order on AI Safety requires companies to audit algorithms for bias, while edge AI gains traction for processing data locally on devices. This shift not only reduces latency but also minimizes cloud dependency—a win for industries handling sensitive data like finance and defense.  

---

## Section 2: Key Players Driving Innovation  
### **Tech Giants and Startup Disruptors**  
OpenAI’s ChatGPT and Google’s Bard dominate conversational AI, but NVIDIA’s H100 GPUs power most large language models behind the scenes. Startups like Anthropic (creator of Claude AI) challenge incumbents with ethical AI frameworks, while Stability AI’s open-source models empower developers in niche sectors like architectural design.  

### **Collaborative Ecosystems**  
Innovation thrives through partnerships. The IBM-Meta AI Alliance promotes open-source research, while platforms like Hugging Face democratize access to pre-trained models. Academic institutions play a crucial role too—MIT’s CSAIL recently released a framework for auditing AI hiring tools, addressing biases in recruitment algorithms.  

---

## Section 3: Noteworthy News & Developments  
### **Capital Moves and Policy Shifts**  
Microsoft’s $10 billion OpenAI investment and Amazon’s Bedrock launch highlight corporate bets on generative AI. Governments keep pace: China implemented strict AI governance rules, while the U.S. allocated $1.6 billion for AI safety research in 2024’s budget.  

### **Hardware Leaps Forward**  
Tesla’s Optimus Gen-2 robot demonstrates unprecedented environmental interaction capabilities, potentially revolutionizing manufacturing. Meanwhile, startups like Cerebras Systems challenge traditional chip design with wafer-scale processors optimized for AI workloads.  

---

## Section 4: Ethical Considerations & Challenges  
### **Bias Mitigation in Practice**  
Facial recognition systems still show 10-15% higher error rates for darker-skinned women (MIT Media Lab). In response, cities like Portland now ban their use in public spaces. Companies like Pymetrics counter this by developing auditable AI hiring tools that explain algorithmic decisions to candidates.  

### **Job Market Evolution**  
While fears of AI-driven unemployment persist, the World Economic Forum predicts 97 million new AI-augmented roles by 2025. Reskilling programs gain momentum—AT&T’s "Future Ready" initiative has already trained 250,000 employees in AI literacy since 2022.  

---

## Conclusion  
AI’s potential is limitless, but its success hinges on balancing innovation with responsibility. As Stanford HAI researcher Fei-Fei Li notes: "AI should reflect the diversity of human values, not amplify our biases." From healthcare breakthroughs to sustainable practices, progress requires collaboration across sectors.  

---

**Call to Action**  
Stay ahead of the curve! [Subscribe to our AI Insights newsletter](https://example.com/newsletter) for monthly deep dives. **Download our free [2023 AI Adoption Checklist](https://example.com/checklist)** to streamline your business’s integration strategy.  

*For further reading, explore our guide on [AI in Finance: Regulatory Challenges and Opportunities](https://example.com/finance-ai).*  

*Keywords: Latest AI trends 2023, AI applications in business, Ethical AI best practices, Generative AI tools for developers, AI in healthcare advancements*